## TFRecord full 용 derectory 만들기 (2:1로 분리)

In [1]:
import os, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

df_train = pd.read_csv('./data/train_df.csv')
df_test = pd.read_csv('./data/test_df.csv')

labelList = df_train['label'].unique()
labelList

original_dataset_dir = './data/train'

base_dir = 'data/train_full'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir,'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir,'validation')
os.mkdir(validation_dir)

for d_path in [train_dir, validation_dir]:

    for l_name in labelList:    
        label_df = df_train[df_train['label']==l_name].reset_index()
        
        for idx, l_row in label_df.iterrows() :
            src = os.path.join(original_dataset_dir, l_row.file_name).replace('\\','/')
            
            if idx <= len(label_df)//3:
                dst = os.path.join(validation_dir, l_row.file_name).replace('\\','/')
            else :
                dst = os.path.join(train_dir, l_row.file_name).replace('\\','/')
                
            shutil.copyfile(src,dst)

In [9]:
path_dir = './data/train_ex'
label_list = sorted(os.listdir(path_dir))
label_list

['bottle-broken_large',
 'bottle-broken_small',
 'bottle-contamination',
 'bottle-good',
 'cable-bent_wire',
 'cable-cable_swap',
 'cable-combined',
 'cable-cut_inner_insulation',
 'cable-cut_outer_insulation',
 'cable-good',
 'cable-missing_cable',
 'cable-missing_wire',
 'cable-poke_insulation',
 'capsule-crack',
 'capsule-faulty_imprint',
 'capsule-good',
 'capsule-poke',
 'capsule-scratch',
 'capsule-squeeze',
 'carpet-color',
 'carpet-cut',
 'carpet-good',
 'carpet-hole',
 'carpet-metal_contamination',
 'carpet-thread',
 'grid-bent',
 'grid-broken',
 'grid-glue',
 'grid-good',
 'grid-metal_contamination',
 'grid-thread',
 'hazelnut-crack',
 'hazelnut-cut',
 'hazelnut-good',
 'hazelnut-hole',
 'hazelnut-print',
 'leather-color',
 'leather-cut',
 'leather-fold',
 'leather-glue',
 'leather-good',
 'leather-poke',
 'metal_nut-bent',
 'metal_nut-color',
 'metal_nut-flip',
 'metal_nut-good',
 'metal_nut-scratch',
 'pill-color',
 'pill-combined',
 'pill-contamination',
 'pill-crack',
 'p

## 하나의 TFRecord만들기

In [10]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm.notebook import tqdm
from PIL import Image
import io


############ Tensorflow에서 제공된 Type별 Feature 생성 코드 ############

def _bytes_feature(value):
    # string / byte 타입을 받아서 byte list를 리턴.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    # float / double 타입을 받아서 float list를 리턴
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _floatarray_feature(array):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=array))

def _int64_feature(value):
    # bool / enum / int / uint 타입을 받아서 int64 list를 리턴
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


############ path ############        

df_train = pd.read_csv('./data/train_df.csv')
df_test = pd.read_csv('./data/test_df.csv')

train_src = './data/train_full/train/'
valid_src = './data/train_full/validation/'
test_src = './data/test/'

path_dir = './data/train_ex'
label_list = sorted(os.listdir(path_dir)) # label 리스트
label_dict = { l_name: n for n, l_name in enumerate(label_list)}

############ DataFrame 생성 ############   

def labeling(img_name):
    img_label = df_train[df_train['file_name']==img_name].iloc[0]['label']
    return label_dict[img_label]

train_data_df = pd.DataFrame(os.listdir(train_src),
                             columns=['filename'])
valid_data_df = pd.DataFrame(os.listdir(valid_src),
                             columns=['filename'])

train_data_df['label'] = train_data_df['filename'].map(lambda x : labeling(x))
valid_data_df['label'] = valid_data_df['filename'].map(lambda x : labeling(x))


train_data_df = train_data_df.astype({'label': 'int'})
valid_data_df = valid_data_df.astype({'label': 'int'})


train_data_df['filename']  = train_data_df['filename'].map(lambda x : train_src + x)
valid_data_df['filename']  = valid_data_df['filename'].map(lambda x : valid_src + x)


display(train_data_df)
display(valid_data_df)

2022-05-02 15:28:37.940110: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


,filename,label
0,./data/train_full/train/11951.png,63
1,./data/train_full/train/12833.png,21
2,./data/train_full/train/13756.png,52
3,./data/train_full/train/12906.png,40
4,./data/train_full/train/11836.png,33
...,...,...
2783,./data/train_full/train/11684.png,72
2784,./data/train_full/train/12771.png,76
2785,./data/train_full/train/13549.png,28
2786,./data/train_full/train/13895.png,28


,filename,label
0,./data/train_full/validation/10489.png,21
1,./data/train_full/validation/10169.png,59
2,./data/train_full/validation/10708.png,45
3,./data/train_full/validation/11545.png,28
4,./data/train_full/validation/11166.png,55
...,...,...
1484,./data/train_full/validation/10149.png,40
1485,./data/train_full/validation/10517.png,21
1486,./data/train_full/validation/10801.png,72
1487,./data/train_full/validation/11947.png,66


In [11]:
############ TFRecord 생성 함수 정의 ############        
        
def to_tfrecords(id_list, label_list, tfrecords_name):
    # id_list : 이미지파일명을 가지고 있는 list
    # label_list : 이미지파일의 label을 가지고 있는 list
    # tfrecords_name : tfrecord의 이름(train,validation 구별을 위해필요)
    
    print("Converting 시작 (" + os.path.join(tfrecords_name + '.tfrecords') + ' 파일생성)')
    # GZIP으로 압축한 TFRecord 생성하기 위한 option
    options = tf.io.TFRecordOptions(compression_type = 'GZIP')
        
    with tf.io.TFRecordWriter(path=os.path.join(tfrecords_name + '.tfrecords'), 
                              options=options) as writer:
        
        # tqdm은 반복문의 진행상황을 progressbar를 보여주는 모듈
        # tqdm 파라미터

        # iterable : 반복자 객체
        # desc : 진행바 앞에 텍스트 출력
        # total : int, 전체 반복량
        # leave : bool, default로 True. (진행상태 잔상이 남음)
        # ncols : 진행바 컬럼길이. width 값으로 pixel 단위로 보임.
        # mininterval, maxinterval : 업데이트 주기. 
        #                            기본은 mininterval=0.1 sec, maxinterval=10 sec
        # miniters : Minimum progress display update interval, in iterations.
        # ascii : True로 하면 '#'문자로 진행바가 표시됨.
        # initial : 진행 시작값. 기본은 0
        # bar_format : str

        # tqdm method
        # clear() : 삭제
        # refresh() : 강제 갱신
        
        for id_, label_ in tqdm(zip(id_list, label_list), 
                                total=len(id_list), 
                                position=0, 
                                leave=True):
            image_path = id_
            
            # TFRecord를 생성하는 시점에 resize를 할수도 있고 TFRecord를 읽어서 DataSet을 만들때
            # resize를 할 수 도 있다. 
            # resize 된 이미지를 저장할 경우 처리에 주의해야 한다. 
            # 잘못하는 경우 이미지 데이터가 깨져서 저장될 수 있음.
            # 일반적으로 원본을 저장하고 사용할 때 resize해서 사용하는것이 일반적임.
            
            # 원본 이미지를 resize하지 않고 TFRecord로 저장.
            _binary_image = tf.io.read_file(image_path)
            
            # Example 객체 생성
            # 파이썬의 문자열은 모두 unicode. unicode 문자열을 byte배열로 바꾸는 함수가 encode()
            # 각 type에 맞게 Feature 객체 생성 후 dict 생성. 
            # 이 dict를 이용해 Feauture 객체 생성 후 Example 객체 생성.
            string_set = tf.train.Example(features=tf.train.Features(feature={
                'image_raw': _bytes_feature(_binary_image),
                'label': _int64_feature(label_)                
            }))

            # 만들어진 Example 객체를 binary string으로 변환한 후 파일에 저장
            writer.write(string_set.SerializeToString())    
            

In [12]:
############ 데이터 분리 및 TFRecord 생성 함수 호출 ############        

from sklearn import utils

# train_ids, val_ids, train_labels, val_labels = \
#     train_test_split(df['filename'], 
#                      df['label'], 
#                      test_size=0.2, 
#                      random_state=40, 
#                      shuffle=True)


# to_tfrecords(train_ids, train_labels, './cat_dog_train')
# to_tfrecords(val_ids, val_labels, './cat_dog_valid')

train_shuffle_df = utils.shuffle(train_data_df)
valid_shuffle_df = utils.shuffle(valid_data_df)

data_dir = './data/class_tf'
os.mkdir(data_dir)

to_tfrecords(train_shuffle_df['filename'], train_shuffle_df['label'], './data/class_tf/train_0502')
to_tfrecords(valid_shuffle_df['filename'], valid_shuffle_df['label'], './data/class_tf/valid_0502')

Converting 시작 (./data/class_tf/train_0502.tfrecords 파일생성)


  0%|          | 0/2788 [00:00<?, ?it/s]

2022-05-02 15:28:42.623622: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-02 15:28:42.633991: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-02 15:28:42.678709: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 15:28:42.679055: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-05-02 15:28:42.679088: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-05-02 15:28:42.684820: I tensorflow/stream_executor/platform/default/dso_loade

Converting 시작 (./data/class_tf/valid_0502.tfrecords 파일생성)


  0%|          | 0/1489 [00:00<?, ?it/s]

## TFRecord에서 데이터 읽기

In [ ]:
def readRecord(filename_queue):
    reader = tf.TFRecordReader()
    _.serialized_example = reader.read(file_name_queue)
    
    